In [1]:
!wget https://ndownloader.figshare.com/files/11767817 -O /data/joe/misinformation/pheme.tar.bz2
!tar -xf /data/joe/misinformation/pheme.tar.bz2com/files/11767946 -O pheme_preprocess.py

--2020-11-12 20:11:33--  https://ndownloader.figshare.com/files/11767817
Resolving ndownloader.figshare.com (ndownloader.figshare.com)... 63.33.10.25, 34.255.40.113, 34.246.108.234, ...
Connecting to ndownloader.figshare.com (ndownloader.figshare.com)|63.33.10.25|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/11767817/PHEME_veracity.tar.bz2 [following]
--2020-11-12 20:11:33--  https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/11767817/PHEME_veracity.tar.bz2
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.88.251
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.88.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46529729 (44M) [binary/octet-stream]
Saving to: ‘./data/pheme.tar.bz2’

./data/pheme.tar.bz 100%[===================>]  44.37M  2.36MB/s    in 23s     

2020-11-12 20:11:57 (1.95 MB/s) - ‘./data/pheme.tar.bz2’

In [30]:
import json
from pheme_preprocess import convert_annotations
import os
import pandas as pd
from sklearn.model_selection import train_test_split

DATA_DIR = "/data/joe/misinformation/all-rnr-annotated-threads"
events = os.listdir(DATA_DIR)
all_data = []
split = []
for e in events:
    data = []
    if e[0] == ".": continue
    tweets = os.listdir(os.path.join(DATA_DIR, e, 'rumours'))
    for t in tweets:
        if t[0] == ".": continue
        tweet_dir = os.path.join(DATA_DIR, e, 'rumours', t)
        annotation = json.load(open(os.path.join(tweet_dir, 'annotation.json')))
        label = convert_annotations(annotation, string=True)
        if label in ["true","false","unverified"]:
            source = json.load(open(os.path.join(tweet_dir, 'source-tweets', t+".json")))
            data.append({"event":e, "text":source["text"], "id":t, "label":label})
    train, test = train_test_split(data, test_size=0.20, random_state=16377)
    train, dev  = train_test_split(train, test_size=0.1/8, random_state=16377)
    split.extend(["train"]*len(train))
    split.extend(["dev"]*len(dev))
    split.extend(["test"]*len(test))

    all_data.extend(train)
    all_data.extend(dev)
    all_data.extend(test)
    
df = pd.DataFrame(all_data)
df['exp_split'] = split
#df["label"] = 1 - df["label"] # flip label. label == 1 means containing misinformation

In [34]:
df.to_csv("/data/joe/misinformation/pheme_misinformation_all.csv")

In [36]:
df_drop_unverified = df[df["label"]!="unverified"]
df_drop_unverified.to_csv("/data/joe/misinformation/pheme_misinformation_drop_unverified.csv")
df_drop_unverified

,event,text,id,label,exp_split
0,gurlitt-all-rnr-threads,Swiss museum will accept Gurlitt art trove htt...,536844209982959616,true,train
1,gurlitt-all-rnr-threads,#Gurlitt's art collection is going to a Bern m...,536827540829384704,true,train
2,gurlitt-all-rnr-threads,Summary of the #Gurlitt agreement between Kuns...,536848134681690112,true,train
3,gurlitt-all-rnr-threads,#BREAKING - Museum of Fine Arts Bern to accept...,536825848624865280,true,train
4,gurlitt-all-rnr-threads,Bern Fine Arts Museum says yes to controversia...,536827886926565376,true,train
...,...,...,...,...,...
2397,ebola-essien-all-rnr-threads,BREAKING: Unconfirmed reports claim AC Milan m...,521350916872876033,false,train
2398,ebola-essien-all-rnr-threads,Micheal Essien denying the Ebola rumours like ...,521369179392581632,false,dev
2399,ebola-essien-all-rnr-threads,Essien and his lawyers are considering to file...,522286703848218624,false,test
2400,ebola-essien-all-rnr-threads,AC Milan midfielder Michael Essien has been di...,521360486387175424,false,test
